## Runs

투자 전략은 $\text{IID}$ process로부터 추출된 수익률을 생성하는 경우가 드물다. 이런 성질이 없기 때문에 투자 수익률 계열은 빈번한 run을 나타낸다. run은 동일한 부호 수익률의 연속된 시퀀스다. 그 결과 런은 하방 리스크(downside risk)를 증가시키므로 적절한 척도로 평가해야 한다.

### 1. Returns Concentration

베팅으로부터 수익률의 시계열 $\{r_t\}_{t = 1, \dots, T}$가 주어져 있을 때 두 가중치 계열 $w^-$와 $w^+$를 계산한다.

$$
\begin{align}
r^+ =& \{r_t | r_t \geq 0\}_{t=1,\dots,T} \\
r^- =& \{r_t | r_t < 0\}_{t=1,\dots,T} \\
w^+ =& \left\{ r_t^+ \left( \sum_t r_t^+\right)^{-1}\right\} \\
w^- =& \left\{ r_t^- \left( \sum_t r_t^-\right)^{-1}\right\}
\end{align}
$$

Herfindahl-Hirschman Index(HHI)로부터 영감을 받아 $\left\Vert \cdot \right\Vert$가 백터의 크기를 의미할 때 $\left\Vert w^+ \right\Vert > 1$에 대해 양의 수익률 집중도를 다음과 같이 정의할 수 있다.

$$h^+ \equiv \frac{\Sigma_t \left( w_t^+\right)^2 - \left\Vert w^+ \right \Vert^{-1}}{1 - \left\Vert w^+ \right \Vert^{-1}} = \left( \frac{E\left[ (r_t^+)^2\right]}{E\left[r_t^+\right]^2} - 1\right)\left( \left\Vert r^+\right\Vert - 1\right)^{-1}$$

그리고 $\left\Vert w^- \right\Vert > 1$에 대해 동일한 음의 수익률의 집중도를 다음과 같이 정의할 수 있다.

$$h^- \equiv \frac{\Sigma_t \left( w_t^-\right)^2 - \left\Vert w^- \right \Vert^{-1}}{1 - \left\Vert w^- \right \Vert^{-1}} = \left( \frac{E\left[ (r_t^-)^2\right]}{E\left[r_t^-\right]^2} - 1\right)\left( \left\Vert r^-\right\Vert - 1\right)^{-1}$$

Jensen-s Inequality로부터 $E[r_t^+]^2 \leq E[(r_t^+)^2]$라는 것을 알고 있다. 또 $\frac{E[(r_t^+)^2]}{E[r_t^+]^2} \leq \left\Vert r^+ \right\Vert$이므로 음의 베팅 수익률과 동일한 경계를 사용해 $E[r_t^+]^2 \leq E[(r_t^+)^2] \leq E[r_t^+]^2\left\Vert r^+ \right\Vert$를 도출한다. 이런 정의는 몇 가지 흥미로운 성질을 갖고 있다.

1. $0 \leq h^+ \leq 1$
2. $h^+ = 0 \Leftrightarrow w_t^+ = \left\Vert w^+ \right\Vert^{-1}. \forall t$(균등 수익률)
3. $h^+ = 0 \Leftrightarrow \exists i|w_t^+ = \sum_t w_t^+$(오직 하나의 0이 아닌 수익률)

월별 베팅 $h[t]$의 집중도에 대해서도 이와 비슷한 식을 유도할 수 있다. 아래의 코드는 이런 개념을 구현하고 있다. 베팅의 수익률이 다음을 만족하는 전략에 관심이 있다.

- 높은 sharpe ratio
- 높은 연간 베팅 수 $\left\Vert r^+ \right\Vert + \left\Vert r^- \right\Vert = T$
- 높은 hit ratio(상대적으로 낮은 $\left\Vert r^- \right\Vert$)
- 낮은 $h^+$ (오른쪽 fat tail이 없는 것)
- 낮은 $h^-$ (왼쪽 fat tail이 없는 것)
- 낮은 $h[t]$ (베팅이 시간에 대해 집중되지 않음)

### 2. Drawdown and Timeunder Water

Drawdown은 투자로 연속된 두 최고점 (high-water mark) 사이에서의 최대 손실이고, Time under water는 HWM과 손익이 이전 최대 손익을 초과한 시점 사이의 경과 시간(time elapsed)을 의미한다. 이런 개념들은 아래 코드를 읽어 보면 잘 이해할 수 있다. 이 코드는 수익률 계열, 달러 성과 계열로부터 $\text{DD}$와 $\text{TuW}$를 도출한다. 아래 그림은 $\text{DD}$와 $\text{TuW}$ 예제를 제공한다.

### 3. Runs Statistics for Performance Evaluation

몇몇 유용한 run 통계량의 척도에는 다음과 같은 것이 있다.

- **HHI index on positive returns** : `getHHI(ret[ret >= 0])`이다.
- **HHI index on negative returns** : `getHHI(ret[ret < 0])`이다.
- **HHI index on time between bets** : `getHHI(ret.groupby(pd.TimeGrouper(freq='ME')).count()`이다.
- **95-percentile DD** : 위 코드에서 도출된 $\text{DD}$ 계열의 95번째 백분위이다.
- **95-percentile TuW** : 위 코드에서 도출된 $\text{TuW}$ 계열의 95번째 백분위이다.

## Implementation Shortfall

투자 전략은 종종 거래 구축 비용에 대해 잘못된 가정을 함으로써 실패한다. 이 부분의 주요한 몇 가지 척도에는 다음과 같은 것이 있다.

- **Broker fees per turnover** : 거래한 포트폴리오에 대해 브로커에게 지급하는 비용으로, 환전 비용도 포함한다.
- **Average slippage per turnover** : 하나의 포트폴리오 거래에 관여된 실행 비용으로 브로커 비용을 제외한다. 예를 들어, 주문을 실행 브로커에 보낸 순간의 중간 가격(mid-price)보다 높은 체결가(fill-price)로 증권을 매수했을 때 발생하는 손실을 포함한다.
- **Dollar performance per turnover** : 달러 성과(브로커 비용과 슬리피지 비용을 포함)와 총 포트폴리오 거래액 간의 비율이다. 이 지표는 전략이 손익분기 되기 전까지 실행에 비용이 얼마나 소요될 것인지를 나타낸다.
- **Return on execution costs** : 달러 성과(브로커 비용과 슬리피지 비용 포함)와 총 실행 비용 사이의 비율이다. 전략이 기대 이하 실행에서도 살아남기 위해서는 큰 배수(multiplier)여야 한다.